In [1]:
import pandas as pd
from utils import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split

import numpy as np
# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image

from xgboost import XGBClassifier
import xgboost as xgb

In [2]:
dataset = load_clean_data()
print("done with dataset")
future = create_future_dataset()

done with dataset
14850
14850


In [6]:
dataset.columns 

Index(['match_id', 'tourney_date', 'surface', 'winner_id', 'loser_id', 'w_ace',
       'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms',
       'w_bpSaved', 'w_bpFaced', 'w_minutes', 'date_x', 'w_losses', 'w_wins',
       'w_hand', 'w_ht', 'w_ioc', 'w_rank', 'w_num_matches', 'w_win_pct',
       'opponent_x', 'w_ace_opp', 'w_df_opp', 'w_svpt_opp', 'w_1stIn_opp',
       'w_1stWon_opp', 'w_2ndWon_opp', 'w_SvGms_opp', 'w_bpSaved_opp',
       'w_bpFaced_opp', 'w_minutes_opp', 'w_losses_opp', 'w_wins_opp',
       'w_num_matches_opp', 'w_win_pct_opp', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'l_minutes', 'date_y', 'l_losses', 'l_wins', 'l_hand', 'l_ht', 'l_ioc',
       'l_rank', 'l_num_matches', 'l_win_pct', 'opponent_y', 'l_ace_opp',
       'l_df_opp', 'l_svpt_opp', 'l_1stIn_opp', 'l_1stWon_opp', 'l_2ndWon_opp',
       'l_SvGms_opp', 'l_bpSaved_opp', 'l_bpFaced_opp', 'l_minutes_opp',
       'l_losses_opp', 

In [10]:
dataset['w_id'] = dataset['winner_id']
dataset['l_id'] = dataset['loser_id']

In [11]:
def shuffle_and_rename(df):
    # Create a boolean mask that randomly decides whether to swap winners and losers
    mask = np.random.rand(len(df)) > 0.5

    # Initialize empty DataFrames for player1 and player2
    player1_df = pd.DataFrame()
    player2_df = pd.DataFrame()

    # Assign columns based on the mask
    # If mask is True, winner becomes player1, else loser becomes player1
    player1_df = df[mask].rename(columns=lambda x: x.replace('w_', 'player1_').replace('l_', 'player2_'))
    player2_df = df[~mask].rename(columns=lambda x: x.replace('w_', 'player2_').replace('l_', 'player1_'))

    # Concatenate the two DataFrames back together
    shuffled_df = pd.concat([player1_df, player2_df])

    return shuffled_df

# Apply the function
df_rand = shuffle_and_rename(dataset)

# Shuffle the final DataFrame rows if necessary
#df_rand = df_rand.sample(frac=1).reset_index(drop=True)
df_rand['output'] = df_rand.apply(lambda x: 1 if x['winner_id'] == x['player1_id'] else 0, axis=1)
df_rand.sort_values('match_id', inplace=True)
df_rand.drop(columns=['tourney_date','date_x','date_y'], inplace=True)
# View the result
df_rand.drop_duplicates(inplace=True)
df_rand

,match_id,surface,winner_id,loser_id,player1_ace,player1_df,player1_svpt,player1_1stIn,player1_1stWon,player1_2ndWon,...,player2_bpFaced_opp,player2_minutes_opp,player2_losses_opp,player2_wins_opp,player2_num_matches_opp,player2_win_pct_opp,lower_won,player1_id,player2_id,output
0,2,Hard,200282,111442,7.800000,2.200000,93.000000,57.200000,40.000000,15.000000,...,0.000000,80.000000,0.0,1.0,1.0,1.000000,0,111442,200282,0
1,5,Hard,200282,126203,8.500000,2.111111,81.944444,48.611111,35.833333,15.388889,...,2.000000,70.000000,0.0,1.0,1.0,1.000000,1,126203,200282,0
2,9,Hard,200282,105526,4.000000,2.133333,73.600000,44.800000,32.233333,16.366667,...,5.000000,154.000000,1.0,0.0,1.0,0.000000,0,200282,105526,1
3,10,Hard,200282,126203,4.000000,2.133333,73.600000,44.800000,32.233333,16.366667,...,4.500000,73.500000,2.0,0.0,2.0,0.000000,1,200282,126203,1
4,11,Hard,200282,105138,3.789474,2.289474,76.473684,49.000000,34.947368,14.342105,...,2.000000,78.000000,0.0,1.0,1.0,1.000000,0,105138,200282,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,4822,Hard,206909,104926,3.534884,4.139535,78.581395,45.534884,31.069767,16.302326,...,3.000000,136.000000,0.0,1.0,1.0,1.000000,0,104926,206909,0
2547,4823,Hard,206909,105173,5.857143,1.714286,80.476190,48.809524,35.380952,16.714286,...,7.000000,183.000000,0.0,1.0,1.0,1.000000,0,206909,105173,1
2548,4827,Hard,206909,111442,6.302326,2.162791,83.000000,51.813953,36.534884,14.767442,...,7.333333,136.666667,1.0,2.0,3.0,0.666667,1,111442,206909,0
2549,4830,Hard,206909,105777,5.679487,4.243590,84.410256,53.076923,38.653846,15.307692,...,11.000000,113.000000,1.0,0.0,1.0,0.000000,1,105777,206909,0


In [13]:
dataset.columns

Index(['match_id', 'tourney_date', 'surface', 'winner_id', 'loser_id', 'w_ace',
       'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms',
       'w_bpSaved', 'w_bpFaced', 'w_minutes', 'date_x', 'w_losses', 'w_wins',
       'w_hand', 'w_ht', 'w_ioc', 'w_rank', 'w_num_matches', 'w_win_pct',
       'opponent_x', 'w_ace_opp', 'w_df_opp', 'w_svpt_opp', 'w_1stIn_opp',
       'w_1stWon_opp', 'w_2ndWon_opp', 'w_SvGms_opp', 'w_bpSaved_opp',
       'w_bpFaced_opp', 'w_minutes_opp', 'w_losses_opp', 'w_wins_opp',
       'w_num_matches_opp', 'w_win_pct_opp', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'l_minutes', 'date_y', 'l_losses', 'l_wins', 'l_hand', 'l_ht', 'l_ioc',
       'l_rank', 'l_num_matches', 'l_win_pct', 'opponent_y', 'l_ace_opp',
       'l_df_opp', 'l_svpt_opp', 'l_1stIn_opp', 'l_1stWon_opp', 'l_2ndWon_opp',
       'l_SvGms_opp', 'l_bpSaved_opp', 'l_bpFaced_opp', 'l_minutes_opp',
       'l_losses_opp', 

In [14]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df_rand['player1_ioc_encoded'] = label_encoder.fit_transform(df_rand['player1_ioc'])
df_rand['player2_ioc_encoded'] = label_encoder.fit_transform(df_rand['player2_ioc'])
df_rand['surface_encoded'] = label_encoder.fit_transform(df_rand['surface'])

df_rand.drop(columns=['player1_ioc', 'player1_ioc', 'surface'], axis=1, inplace=True)



In [28]:
from sklearn.preprocessing import LabelEncoder
ioc_encoder = LabelEncoder()
future['w_ioc_encoded'] = ioc_encoder.fit_transform(future['w_ioc'])
future['l_ioc_encoded'] = ioc_encoder.fit_transform(future['l_ioc'])
surface_encoder = LabelEncoder()
future['surface_encoded'] = surface_encoder.fit_transform(future['surface'])

mapping_surface = dict(zip(surface_encoder.classes_, surface_encoder.transform(surface_encoder.classes_)))

future.drop(columns=['player1_ioc', 'player2_ioc', 'surface'], axis=1, inplace=True)

KeyError: 'player1_ioc'

In [17]:
xgb_model_latest = XGBClassifier()
xgb_model_latest.load_model("best_xgboost.json")

In [20]:
future.columns

Index(['player1_id', 'player2_id', 'player1_losses', 'player1_wins',
       'player1_ace', 'player1_df', 'player1_svpt', 'player1_1stIn',
       'player1_1stWon', 'player1_2ndWon', 'player1_SvGms', 'player1_bpSaved',
       'player1_bpFaced', 'player1_minutes', 'player1_hand', 'player1_ht',
       'player1_rank', 'player1_num_matches', 'player1_win_pct',
       'player1_losses_opp', 'player1_wins_opp', 'player1_ace_opp',
       'player1_df_opp', 'player1_svpt_opp', 'player1_1stIn_opp',
       'player1_1stWon_opp', 'player1_2ndWon_opp', 'player1_SvGms_opp',
       'player1_bpSaved_opp', 'player1_bpFaced_opp', 'player1_minutes_opp',
       'opponent_x', 'player1_num_matches_opp', 'player1_win_pct_opp',
       'player2_losses', 'player2_wins', 'player2_ace', 'player2_df',
       'player2_svpt', 'player2_1stIn', 'player2_1stWon', 'player2_2ndWon',
       'player2_SvGms', 'player2_bpSaved', 'player2_bpFaced',
       'player2_minutes', 'player2_hand', 'player2_ht', 'player2_ioc',
       'pla

In [57]:
future.head()

,player1_id,player2_id,player1_losses,player1_wins,player1_ace,player1_df,player1_svpt,player1_1stIn,player1_1stWon,player1_2ndWon,...,player2_SvGms_opp,player2_bpSaved_opp,player2_bpFaced_opp,player2_minutes_opp,opponent_y,player2_num_matches_opp,player2_win_pct_opp,player1_ioc_encoded,player2_ioc_encoded,surface_encoded
0,206173,100644,36.0,106.0,5.855072,2.072464,77.543478,46.905797,34.992754,16.652174,...,17.0,5.40,9.000,180.400,206173,5.0,0.8,21,17,2
1,206173,100644,8.0,4.0,6.083333,3.166667,88.666667,51.666667,37.833333,19.750000,...,17.0,5.40,9.000,180.400,206173,5.0,0.8,21,17,1
2,206173,100644,16.0,30.0,3.227273,2.090909,80.000000,47.431818,33.727273,17.272727,...,17.0,5.40,9.000,180.400,206173,5.0,0.8,21,17,0
3,206173,207989,36.0,106.0,5.855072,2.072464,77.543478,46.905797,34.992754,16.652174,...,15.0,4.25,8.125,166.875,206173,8.0,0.5,21,13,2
4,206173,207989,8.0,4.0,6.083333,3.166667,88.666667,51.666667,37.833333,19.750000,...,15.0,4.25,8.125,166.875,206173,8.0,0.5,21,13,1


In [50]:
# Split the data into features (X) and target (y)
X = future.drop(['player1_id', 'player2_ioc', 'player2_id', 'opponent_x', 'opponent_y'], axis=1)
X = X[['player1_ace', 'player1_df', 'player1_svpt', 'player1_1stIn', 'player1_1stWon', 'player1_2ndWon', 'player1_SvGms', 
       'player1_bpSaved', 'player1_bpFaced', 'player1_minutes', 'player1_losses', 'player1_wins', 'player1_hand', 'player1_ht', 
       'player1_rank', 'player1_num_matches', 'player1_win_pct', 'player1_ace_opp', 'player1_df_opp', 'player1_svpt_opp', 
       'player1_1stIn_opp', 'player1_1stWon_opp', 'player1_2ndWon_opp', 'player1_SvGms_opp', 'player1_bpSaved_opp', 
       'player1_bpFaced_opp', 'player1_minutes_opp', 'player1_losses_opp', 'player1_wins_opp', 'player1_num_matches_opp', 
       'player1_win_pct_opp', 'player2_ace', 'player2_df', 'player2_svpt', 'player2_1stIn', 'player2_1stWon', 'player2_2ndWon', 
       'player2_SvGms', 'player2_bpSaved', 'player2_bpFaced', 'player2_minutes', 'player2_losses', 'player2_wins', 'player2_hand', 
       'player2_ht', 'player2_rank', 'player2_num_matches', 'player2_win_pct', 'player2_ace_opp', 'player2_df_opp', 'player2_svpt_opp', 
       'player2_1stIn_opp', 'player2_1stWon_opp', 'player2_2ndWon_opp', 'player2_SvGms_opp', 'player2_bpSaved_opp', 'player2_bpFaced_opp', 
       'player2_minutes_opp', 'player2_losses_opp', 'player2_wins_opp', 'player2_num_matches_opp', 'player2_win_pct_opp', 
       'player1_ioc_encoded', 'player2_ioc_encoded', 'surface_encoded']]

# Split the data into training and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

ddata = xgb.DMatrix(X)

In [51]:
predictions = xgb_model_latest.predict(X)

In [74]:
match_preds = pd.concat([future, pd.DataFrame(predictions)], axis=1)
match_preds['pred_winner_id'] = match_preds.apply(lambda x: x['player1_id'] if x[0] == 1 else x['player2_id'], axis=1)

In [77]:
match_preds = match_preds[['player1_id', 'player2_id', 'surface_encoded', 'pred_winner_id',]]


In [66]:
ranks = pd.read_csv("atp_100_withIDs.csv")

In [71]:
match_preds = match_preds.merge(ranks[['name', 'id']], left_on='player1_id', right_on='id', how='left').drop('id', axis=1)
match_preds.rename(columns={'name': 'player1_name'}, inplace=True)
match_preds = match_preds.merge(ranks[['name', 'id']], left_on='player2_id', right_on='id', how='left').drop('id', axis=1)
match_preds.rename(columns={'name': 'player2_name'}, inplace=True)
match_preds = match_preds.merge(ranks[['name', 'id']], left_on='pred_winner_id', right_on='id', how='left').drop('id', axis=1)
match_preds.rename(columns={'name': 'pred_winner_name'}, inplace=True)


In [72]:
match_preds

,player1_id,player2_id,pred_winner_id,player1_name,player2_name,pred_winner_name
0,206173,100644,206173,Jannik Sinner,Alexander Zverev,Jannik Sinner
1,206173,100644,100644,Jannik Sinner,Alexander Zverev,Alexander Zverev
2,206173,100644,100644,Jannik Sinner,Alexander Zverev,Alexander Zverev
3,206173,207989,207989,Jannik Sinner,Carlos Alcaraz,Carlos Alcaraz
4,206173,207989,207989,Jannik Sinner,Carlos Alcaraz,Carlos Alcaraz
...,...,...,...,...,...,...
14845,205734,133430,133430,Thiago Seyboth Wild,Denis Shapovalov,Denis Shapovalov
14846,205734,133430,205734,Thiago Seyboth Wild,Denis Shapovalov,Thiago Seyboth Wild
14847,106432,133430,133430,Borna Coric,Denis Shapovalov,Denis Shapovalov
14848,106432,133430,133430,Borna Coric,Denis Shapovalov,Denis Shapovalov
